The aim is to turn useful notebook code cells into reusable python files!

In [1]:
import os
os.getcwd()

'/Users/tituslim/Documents/Personal Learning Folder/Data Science/15. PyTorch Developer Class/symmetrical-octo-spork/Notebooks'

Create datasets and dataloaders using `%%writefile` magic command

In [2]:
# Create directory for going_modular
import os
if not os.path.exists("../going_modular"):
    os.makedirs("../going_modular")
else:
    print("Directory exists")

In [1]:
%%writefile ../going_modular/get_data.py
"""Extracts pizza steak sushi data file
"""
import os
from pathlib import Path
import requests
import zipfile

def main(url: str,
         data_path: str = "data",
         image_dir: str = "pizza_steak_sushi"):
    
    data_path = Path("data/")
    image_path = data_path/image_dir

    if image_path.is_dir():
        print(f"{image_path} directory already exists. Skipping download...")
    else:
        print(f"Creating {image_path} directory...")
        image_path.mkdir(parents=True, exist_ok=True)

    # Download data
    with open(data_path/"pizza_steak_sushi.zip", "wb") as f:
        request = requests.get(url)
        print("Downloading pizza_steak_sushi data...")
        f.write(request.content)

    # Unzip zip folder
    with zipfile.ZipFile(data_path/"pizza_steak_sushi.zip", "r") as zip_ref:
        print("Unziping pizza_steak_sushi data...")
        zip_ref.extractall(image_path)
    
    return {"status": "Data extracted successfully"}

Overwriting ../going_modular/get_data.py


In [2]:
%%writefile ../going_modular/data_setup.py
"""
Contains functionality for creating PyTorch DataLoaders for
image classification tasks.
"""
import os
import sys
from typing import Tuple, List
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

NUM_WORKERS = os.cpu_count()

def create_dataloaders(
    train_dir: str,
    test_dir: str,
    transform: transforms.Compose,
    batch_size: int,
    num_workers: int = NUM_WORKERS
) -> Tuple[torch.utils.data.DataLoader, 
           torch.utils.data.DataLoader,
           List]:
    """Creates torch datasets and subsequently dataloaders for training
    and testing sets. 

    Args:
        train_dir (str): Filepath to train data
        test_dir (str): Filepath to test data
        transform (transforms.Compose): torch transforms.Compose object
        batch_size: Number of samples per batch in each of the datalaoders
        num_workers (int, optional): Defaults to NUM_WORKERS.
    
    Returns:
        Tuple of (train_dataloader, test_dataloader, class_names) where
        classnames is a list of the target classes.
    """
    train_data = datasets.ImageFolder(root = train_dir,
                                      transform = transform)
    class_names = train_data.classes
    test_data = datasets.ImageFolder(root = test_dir,
                                      transform = transform)
    
    train_dataloader = DataLoader(train_data,
                                  batch_size = batch_size,
                                  shuffle = True,
                                  num_workers = num_workers,
                                  pin_memory = True
                                  )
    
    test_dataloader = DataLoader(test_data,
                                  batch_size = batch_size,
                                  shuffle = False,
                                  num_workers = num_workers,
                                  pin_memory = True
                                  )
    
    return train_dataloader, test_dataloader, class_names

Overwriting ../going_modular/data_setup.py


Test data_setup.py

In [2]:
import sys
sys.path.append("../")

In [3]:
from going_modular import data_setup
from torchvision import transforms

train_dir = "./data/pizza_steak_sushi/train/"
test_dir = "./data/pizza_steak_sushi/test/"

transform = transforms.Compose([
    transforms.Resize(size = (224,224)),
    transforms.ToTensor()
])

train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir = train_dir,
    test_dir = test_dir,
    transform = transform,
    batch_size = 32
)

class_names

['pizza', 'steak', 'sushi']

In [3]:
%%writefile ../going_modular/model_builder.py
"""Contains pytorch code to develop TinyVGG architecture
"""

import torch
from torch import nn

class TinyVGG(nn.Module):
    """
    Model architecture copying TinyVGG from: 
    https://poloclub.github.io/cnn-explainer/
    """
    def __init__(self, 
                 num_channels: int = 3, 
                 hidden_units: int = 10,
                 num_classes: int = 3):
        """Class constructor

        Args:
            num_channels (int): Number of color channels
            hidden_units (int): Number of hidden units in model
            num_classes (int): Number of labels. Set to 3 for pizza, steak, sushi
        """
        super().__init__()
        self.conv_block_1 = nn.Sequential(
            nn.Conv2d(in_channels=num_channels, 
                      out_channels=hidden_units, 
                      kernel_size=3, # how big is the square that's going over the image?
                      stride=1, # default
                      padding=1), # options = "valid" (no padding) or "same" (output has same shape as input) or int for specific number 
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units, 
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,
                         stride=2) # default stride value is same as kernel_size
        )
        self.conv_block_2 = nn.Sequential(
            nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            # Where did this in_features shape come from? 
            # It's because each layer of our network compresses and changes the shape of our inputs data.
            nn.Linear(in_features=hidden_units*16*16,
                      out_features=num_classes)
        )
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """Overrides forward method from parent. Runs a forward pass

        Args:
            x (torch.Tensor): Image data in tensor format

        Returns:
            torch.Tensor: Logits
        """
        # x = self.conv_block_1(x)
        # # print(x.shape)
        # x = self.conv_block_2(x)
        # # print(x.shape)
        # x = self.classifier(x)
        # # print(x.shape)
        return self.classifier(self.conv_block_2(self.conv_block_1(x))) # <- leverage the benefits of operator fusion

class TinyVGG2(nn.Module):
  def __init__(self, num_color_channels: int, hidden_units: int, num_classes: int):
    super().__init__()
    self.conv_block_1 = nn.Sequential(
        nn.Conv2d(in_channels = num_color_channels,
                  out_channels = hidden_units,
                  kernel_size = 3,
                  stride = 1,
                  padding = 1),
        nn.ReLU(),
        nn.Conv2d(in_channels = hidden_units,
                  out_channels = hidden_units,
                  kernel_size = 3,
                  stride = 1,
                  padding = 1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2,
                     stride = 2)
    )
    self.conv_block_2 = nn.Sequential(
        nn.Conv2d(hidden_units, hidden_units, kernel_size = 3, padding = 1),
        nn.ReLU(),
        nn.Conv2d(hidden_units, hidden_units, kernel_size = 3, padding =1),
        nn.ReLU(),
        nn.MaxPool2d(2)
    )
    self.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features = hidden_units * 56 * 56,
                  out_features = num_classes)
    )
  def forward(self, x: torch.Tensor):
      return self.classifier(self.conv_block_2(self.conv_block_1(x)))

Overwriting ../going_modular/model_builder.py


In [ ]:
import torch
from torchinfo import summary
from going_modular import model_builder
sys.path.append(os.path.join(os.getcwd(),
                             "../src"))
from utils import get_device
device = get_device()

torch.manual_seed(42)
torch.mps.manual_seed(42)
model = model_builder.TinyVGG2(num_color_channels=3,
                               hidden_units = 20,
                               num_classes = len(class_names)).to(device)
summary(model, input_size = [32, 3, 224, 224])

In [ ]:
img, label = next(iter(train_dataloader))
img.shape, label.shape

In [ ]:
out = model(img)
type(out), out.shape

Wrting our training and testing step functions into a python script

In [4]:
%%writefile ../going_modular/engine.py
"""Contains functions for training and testing a PyTorch model
"""

import torch
from torch import nn
from typing import Dict, List, Tuple
from tqdm.auto import tqdm
from collections import defaultdict

def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               device: str) -> Tuple[float, float]:
    """Helper function to train pytorch model on device
    and acquire training metrics per epoch

    Args:
        model (torch.nn.Module): instantiated torch model
        dataloader (torch.utils.data.DataLoader)
        loss_fn (torch.nn.Module)
        optimizer (torch.optim.Optimizer) 
        device (str): Torch device

    Returns:
        Average training loss and training accuracy per epoch
    """
    
    train_loss, train_acc = 0,0
    model.train()

    for batch, (X, y) in enumerate(dataloader):
        
        # Forward pass
        X, y = X.to(device), y.to(device)
        y_pred = model(X) #logits
        loss = loss_fn(y_pred, y)
        train_loss += loss.item()

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Compute metric across all batches
        y_pred_class = torch.argmax(
            torch.softmax(y_pred, dim = 1),
            dim = 1
        )

        train_acc += (y_pred_class==y).sum().item()/len(y_pred)
    
    # Adjust metrics to get average loss and accuracy per batch
    train_loss = train_loss / len(dataloader)
    train_acc = train_acc /len(dataloader)

    return train_loss, train_acc

def test_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              device: str) -> Tuple[float, float]:
    """Runs inference of trained model on test dataset per epoch
    and monitors model test metrics.

    Args:
        model (torch.nn.Module): instantiated torch model
        dataloader (torch.utils.data.DataLoader)
        loss_fn (torch.nn.Module)
        device (str, optional): _description_. Defaults to device.

    Returns:
        Average test loss and test accuracy per epoch
    """
    
    test_loss, test_acc = 0,0
    model.eval()
    
    with torch.inference_mode():
        for batch, (X, y) in enumerate(dataloader):
            
            # Forward pass
            X, y = X.to(device), y.to(device)
            test_pred = model(X) #logits

            # Compute metrics 
            loss = loss_fn(test_pred, y)
            test_loss += loss.item()
            test_pred_class = torch.argmax(
                torch.softmax(test_pred, dim = 1),
                dim = 1
            )
            test_acc += (test_pred_class == y).sum().item()/len(test_pred_class)
    
    # Adjust metrics to get average loss and accuracy per batch
    test_loss = test_loss/len(dataloader)
    test_acc = test_acc/len(dataloader)
    
    return test_loss, test_acc

def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          epochs: int,
          device: str,
          loss_fn: torch.nn.Module = nn.CrossEntropyLoss()): 
    """Wrapper function to train model over specified number of epochs,
    model, dataloaders, optimizer and loss function.

    Args:
        model (torch.nn.Module): instantiated torch model
        train_dataloader (torch.utils.data.DataLoader)
        test_dataloader (torch.utils.data.DataLoader)
        optimizer (torch.optim.Optimizer)
        epochs (int): Number of epochs for training
        loss_fn (torch.nn.Module, optional) Defaults to nn.CrossEntropyLoss().

    Returns:
        Dictionary of results
    """
    
    # Create storage results dictionary
    results = defaultdict(list)

    # Loop through training and testing steps for number of epochs
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model = model,
                                           dataloader = train_dataloader,
                                           loss_fn = loss_fn,
                                           optimizer = optimizer,
                                           device = device)
        test_loss, test_acc = test_step(model = model,
                                        dataloader = test_dataloader,
                                        loss_fn = loss_fn,
                                        device = device)
        print(
            f"Epoch: {epoch+1} | "
            f"train_loss: {train_loss:.4f} | "
            f"train_acc: {train_acc:.4f} | "
            f"test_loss: {test_loss:.4f} | "
            f"test_acc: {test_acc:.4f}"
        )

        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)
    
    return results

Overwriting ../going_modular/engine.py


In [9]:
from going_modular import engine
print("Train and test functions imported!")

Train and test functions imported!


Turning our utility functions for saving a model into a python script

In [5]:
%%writefile ../going_modular/utils.py
"""Utility functions for torch driven computer vision projects
"""

from pathlib import Path
import torch

def save_model(root_dir: str,
               model_name: str,
               model: torch.nn.Module):
    """Saves a torch model to the given root directory

    Args:
        root_dir (str): Name of directory to store model
        model_name (str): Name of the model
        model (torch.nn.Module): Torch model object
    """
    MODEL_PATH = Path(root_dir)
    MODEL_PATH.mkdir(parents=True,
                     exist_ok = True)
    MODEL_NAME = model_name
    MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

    torch.save(obj = model.state_dict(),
            f = MODEL_SAVE_PATH)
    print(f"Saved model to: {MODEL_SAVE_PATH}")

Overwriting ../going_modular/utils.py


In [11]:
from going_modular import utils
print("Utils imported!")

Utils imported!


Create train.py 

In [ ]:
import sys
sys.path.append("../")
from src.utils import get_device
device = get_device()
device

In [6]:
%%writefile ../going_modular/train.py
"""Trains a PyTorch tinyVGG image classification model
using device agnostic code
"""
import os
import get_data, data_setup, engine, model_builder, utils

import sys
sys.path.append("../")
from src.utils import get_device

import torch
from torch import nn
from torchvision import transforms
from timeit import default_timer as timer

import argparse

parser = argparse.ArgumentParser()
parser.add_argument("--seed",
                    type = int,
                    required = False,
                    help = "Random seed for reproducible experiments")
parser.add_argument("--epochs",
                    type = int,
                    required = True,
                    help = "Total number of epochs for training")
parser.add_argument("--batch_size",
                    type = int,
                    required = False,
                    default = 32,
                    help = "Batch size for model training")
parser.add_argument("--lr",
                    type = float,
                    required = True,
                    help = "Learning rate for torch optimizer")
parser.add_argument("--workers",
                    type = int,
                    required = False,
                    default = 0,
                    help = "Number of workers for training")
parser.add_argument("--url",
                    type = str,
                    required = False,
                    default = "https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip",
                    help = "url for data download")

args = parser.parse_args()

#Hyperparameters
BATCH_SIZE = args.batch_size
LEARNING_RATE = args.lr
EPOCHS = args.epochs
NUM_WORKERS = args.workers
URL = args.url
if args.seed:
    SEED = args.seed
    torch.manual_seed(SEED)

HIDDEN_UNITS = 10

#Get data
data_path = "data"
image_dir = "pizza_steak_sushi"
return_ = get_data.main(URL,
                        data_path,
                        image_dir)

#Setup directores
train_dir = f"{data_path}/{image_dir}/train"
test_dir = f"{data_path}/{image_dir}/test"

#Get device
#device = get_device()
device = 'cuda'
if device == "mps":
    torch.mps.manual_seed(SEED)
elif device == 'cuda':
    torch.cuda.manual_seed(SEED)

#Create transforms
data_transform = transforms.Compose([
    transforms.Resize(size = (64,64)),
    transforms.ToTensor()
])

#Create dataloaders and get class names
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir = train_dir,
    test_dir = test_dir,
    transform = data_transform,
    batch_size = BATCH_SIZE,
    num_workers = NUM_WORKERS
)

#Create model
model = model_builder.TinyVGG(
    num_channels = 3,
    hidden_units = HIDDEN_UNITS,
    num_classes = 3
).to(device)

#Setup loss and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),
                             lr = LEARNING_RATE)

#Train model
start_time = timer()
engine.train(model = model,
             train_dataloader = train_dataloader,
             test_dataloader = test_dataloader,
             loss_fn = loss_fn,
             optimizer = optimizer,
             epochs = EPOCHS,
             device = device)
end_time = timer()
print(f"Total training time: {end_time - start_time:.3f} seconds")

#Save model to file
utils.save_model(model = model,
                 root_dir = "models",
                 model_name = "tinyVGG2_pizza_steak_sushi.pth")

print("Model saved in models directory as tinyVGG2_pizza_steak_sushi.pth")

Overwriting ../going_modular/train.py


In [7]:
%run ../going_modular/train.py --seed 42 --epochs 20 --lr 0.001

data/pizza_steak_sushi directory already exists. Skipping download...
Unziping pizza_steak_sushi data...


  0%|          | 0/20 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.1078 | train_acc: 0.2578 | test_loss: 1.1360 | test_acc: 0.2604
Epoch: 2 | train_loss: 1.0847 | train_acc: 0.4258 | test_loss: 1.1620 | test_acc: 0.1979
Epoch: 3 | train_loss: 1.1157 | train_acc: 0.2930 | test_loss: 1.1697 | test_acc: 0.1979
Epoch: 4 | train_loss: 1.0956 | train_acc: 0.4141 | test_loss: 1.1385 | test_acc: 0.1979
Epoch: 5 | train_loss: 1.0985 | train_acc: 0.2930 | test_loss: 1.1428 | test_acc: 0.1979
Epoch: 6 | train_loss: 1.0956 | train_acc: 0.2500 | test_loss: 1.1465 | test_acc: 0.2604
Epoch: 7 | train_loss: 1.1188 | train_acc: 0.3047 | test_loss: 1.1420 | test_acc: 0.2812
Epoch: 8 | train_loss: 1.1038 | train_acc: 0.3438 | test_loss: 1.1099 | test_acc: 0.1979
Epoch: 9 | train_loss: 1.0959 | train_acc: 0.4375 | test_loss: 1.0915 | test_acc: 0.5417
Epoch: 10 | train_loss: 1.0894 | train_acc: 0.4023 | test_loss: 1.0731 | test_acc: 0.5417
Epoch: 11 | train_loss: 1.0736 | train_acc: 0.4102 | test_loss: 1.0504 | test_acc: 0.5729
Epoch: 12 | train_l